In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, applications
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Cargar el dataset CIFAR-10
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# Redimensionar las imágenes a 75x75 (tamaño mínimo para InceptionV3)
def resize_images(images):
    return tf.image.resize(images, [75, 75])

train_images = resize_images(train_images)
test_images = resize_images(test_images)

# Normalizar los valores de píxeles al rango [-1, 1] (preprocesamiento para InceptionV3)
train_images = (train_images / 127.5) - 1
test_images = (test_images / 127.5) - 1

# Convertir las etiquetas a one-hot encoding
train_labels = tf.keras.utils.to_categorical(train_labels, 10)
test_labels = tf.keras.utils.to_categorical(test_labels, 10)

# Cargar InceptionV3 pre-entrenado en ImageNet
base_model = applications.InceptionV3(weights='imagenet',
                                    include_top=False,
                                    input_shape=(75, 75, 3))

# Congelar las capas del modelo base
for layer in base_model.layers:
    layer.trainable = False

# Añadir nuevas capas para nuestro problema
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)  # Reducción dimensional
x = layers.Dense(1024, activation='relu')(x)  # Capa fully-connected
predictions = layers.Dense(10, activation='softmax')(x)  # Capa de salida

# Crear el modelo final
model = Model(inputs=base_model.input, outputs=predictions)

# Compilar el modelo
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Resumen del modelo
model.summary()

# Entrenar el modelo
history = model.fit(train_images, train_labels,
                    batch_size=32,
                    epochs=10,
                    validation_data=(test_images, test_labels))

# Evaluar el modelo
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
print(f'\nPrecisión en el conjunto de prueba: {test_acc:.4f}')

# Descongelar las últimas 20 capas del modelo base
for layer in base_model.layers[-20:]:
    layer.trainable = True

# Recompilar el modelo con una tasa de aprendizaje más baja
model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Continuar el entrenamiento
history_fine = model.fit(train_images, train_labels,
                         batch_size=32,
                         epochs=5,
                         validation_data=(test_images, test_labels))

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 75, 75, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 37, 37,    │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 37, 37,    │         96 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 37, 37,    │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 35, 35,    │      9,216 │ activation[0][0]  │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 35, 35,    │         96 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 35, 35,    │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 35, 35,    │     18,432 │ activation_1[0][… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 35, 35,    │        192 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 35, 35,    │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 17, 17,    │          0 │ activation_2[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 17, 17,    │      5,120 │ max_pooling2d[0]… │
│                     │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 17, 17,    │        240 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 17, 17,    │          0 │ batch_normalizat… │
│ (Activation)        │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 15, 15,    │    138,240 │ activation_3[0][… │
│                     │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 15, 15,    │        576 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_4        │ (None, 15, 15,    │          0 │ batch_normalizat

 Total params: 23,911,210 (91.21 MB)

 Trainable params: 2,108,426 (8.04 MB)

 Non-trainable params: 21,802,784 (83.17 MB)

Epoch 1/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 538s 339ms/step - accuracy: 0.5177 - loss: 1.4224 - val_accuracy: 0.6387 - val_loss: 1.0553
Epoch 2/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 522s 334ms/step - accuracy: 0.6808 - loss: 0.9295 - val_accuracy: 0.6518 - val_loss: 1.0138
Epoch 3/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 528s 338ms/step - accuracy: 0.7280 - loss: 0.8051 - val_accuracy: 0.6536 - val_loss: 1.0137
Epoch 4/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 560s 336ms/step - accuracy: 0.7632 - loss: 0.7015 - val_accuracy: 0.6645 - val_loss: 0.9880
Epoch 5/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 560s 336ms/step - accuracy: 0.8011 - loss: 0.6111 - val_accuracy: 0.6686 - val_loss: 0.9869
Epoch 6/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 521s 333ms/step - accuracy: 0.8355 - loss: 0.5231 - val_accuracy: 0.6681 - val_loss: 0.9945
Epoch 7/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 565s 335ms/step - accuracy: 0.8632 - loss: 0.4580 - val_accuracy: 0.6664 - val_loss: 1.0200
Epoch 8/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 526s 336ms/step - ac